<a href="https://colab.research.google.com/github/TaiseiYYYYMMDD/Image-to-Japanese-Caption/blob/main/Verger_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install -U transformers sentencepiece "pillow<12"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 67.3 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()  # 画像を選択してアップロード

Saving amanda-maria-wZlwKNNKT8w-unsplash.jpg to amanda-maria-wZlwKNNKT8w-unsplash.jpg


In [ ]:
list(uploaded.keys())


['amanda-maria-wZlwKNNKT8w-unsplash.jpg']

In [ ]:
import torch
from PIL import Image
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer

CAPTION_MODEL_NAME = "nlpconnect/vit-gpt2-image-captioning"
device = "cuda" if torch.cuda.is_available() else "cpu"

# モデル読み込み
cap_model = VisionEncoderDecoderModel.from_pretrained(CAPTION_MODEL_NAME).to(device)
cap_processor = ViTImageProcessor.from_pretrained(CAPTION_MODEL_NAME)
cap_tokenizer = AutoTokenizer.from_pretrained(CAPTION_MODEL_NAME)
cap_model.eval()

@torch.no_grad()
def caption_image(image_path: str) -> str:
    image = Image.open(image_path).convert("RGB")
    pixel_values = cap_processor(
        images=image,
        return_tensors="pt"
    ).pixel_values.to(device)

    output_ids = cap_model.generate(
        pixel_values,
        max_length=30,
        num_beams=4,
        no_repeat_ngram_size=2,
        early_stopping=True,
    )

    return cap_tokenizer.decode(
        output_ids[0],
        skip_special_tokens=True
    ).strip()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/982M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [ ]:
image_path = list(uploaded.keys())[0]  # 1枚だけならこれでOK
english_caption = caption_image(image_path)
print("EN:", english_caption)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


EN: a man drinking a glass of water from a bottle


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

TRANS_MODEL_NAME = "facebook/nllb-200-distilled-600M"
SRC_LANG = "eng_Latn"
TGT_LANG = "jpn_Jpan"

tr_tokenizer = AutoTokenizer.from_pretrained(TRANS_MODEL_NAME)
tr_model = AutoModelForSeq2SeqLM.from_pretrained(TRANS_MODEL_NAME).to(device)
tr_model.eval()

@torch.no_grad()
def translate_en_to_ja(text: str) -> str:
    text = (text or "").strip()
    if not text:
        return ""

    tr_tokenizer.src_lang = SRC_LANG
    inputs = tr_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=256
    ).to(device)

    forced_bos_token_id = tr_tokenizer.convert_tokens_to_ids(TGT_LANG)

    outputs = tr_model.generate(
        **inputs,
        forced_bos_token_id=forced_bos_token_id,
        max_length=128,
        num_beams=4,
        early_stopping=True,
    )

    return tr_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
japanese_text = translate_en_to_ja(english_caption)
print("JA:", japanese_text)


JA: 瓶から一杯水を飲む男


-----


In [3]:
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
image_path


Saving OIP.webp to OIP.webp


'OIP.webp'

In [4]:
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

MODEL_NAME = "Salesforce/blip-image-captioning-large"
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = BlipProcessor.from_pretrained(MODEL_NAME)
model = BlipForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
model.eval()

@torch.no_grad()
def caption_image_blip(image_path: str) -> str:
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(device)
    out = model.generate(**inputs, max_new_tokens=30)
    return processor.decode(out[0], skip_special_tokens=True).strip()

caption = caption_image_blip(image_path)
print("EN:", caption)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

EN: there is a cheetah sitting in the grass with its mouth open


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

TRANS_MODEL = "facebook/nllb-200-distilled-600M"
SRC_LANG = "eng_Latn"
TGT_LANG = "jpn_Jpan"
device = "cuda" if torch.cuda.is_available() else "cpu"

tok = AutoTokenizer.from_pretrained(TRANS_MODEL)
mdl = AutoModelForSeq2SeqLM.from_pretrained(TRANS_MODEL).to(device)
mdl.eval()

@torch.no_grad()
def translate_en_to_ja(text: str) -> str:
    tok.src_lang = SRC_LANG
    inputs = tok(text, return_tensors="pt", truncation=True, max_length=256).to(device)
    forced_bos = tok.convert_tokens_to_ids(TGT_LANG)
    out = mdl.generate(**inputs, forced_bos_token_id=forced_bos, max_new_tokens=128, num_beams=4)
    return tok.decode(out[0], skip_special_tokens=True).strip()

print("JA:", translate_en_to_ja(caption))


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

JA: 草の中に口を開いて座っているチーターがいる.
